# Work with a MongoDB docker image

## Docker commands

<mark>**Change the kernel to powershell, before you run the code below.**</mark>

### List all Images in the docker repository

In [ ]:
docker images -a

### Check status of running Images/Containers

In [ ]:
docker ps

## Create a MongoDB container

Download latest version of mongDB

In [ ]:
docker pull mongo

Configure the data-folder on the local-system instead of the docker image

In [ ]:
mkdir -p /mongodata

## Start the MongoDB container

Start the docker image with name **<mark>mongodb</mark>** and mapp the port to 80

In [ ]:
docker run -p 80:27017 -it -v mongodata:/data/db --name mongodb -d mongo
docker ps

### Inspect the container

Run docker inspect **CONTAINR ID**

In [ ]:
$inspected_image = docker inspect 2c20e9dc821c
$object = $inspected_image | ConvertFrom-Json
$object | Select Name, @{l="IPAddress";e={$object.NetworkSettings.IPAddress}}, @{l="Ports";e={($object.NetworkSettings.Ports)}}

### Show the logs of the container

In [ ]:
docker logs mongodb

### Stop and restart MongoDB container

In [ ]:
docker stop mongodb
docker ps -a

In [ ]:
docker start mongodb
docker ps -a

Remove docker container

In [ ]:
docker rm mongodb
docker ps -a

## Python

<mark>**Please change the kernel to python, before you run the code below.**</mark>

### List all databases

Return a list of your system's databases

In [ ]:
def get_dbs(connectionstring):
    '''Connect to MongoDB and add a document'''
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    print(mongo_client.list_database_names())
    mongo_client.close()

connectionstring = "mongodb://0.0.0.0:80"
get_dbs(connectionstring)

### List all collections in a database

Return a list of all collections in your database

In [4]:
def get_collectons(connectionstring, mongodatabase):
    '''Connect to MongoDB and add a document'''
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[mongodatabase]
    print(mongo_db.list_collection_names())
    mongo_client.close()

connectionstring = "mongodb://0.0.0.0:80"
get_collectons(connectionstring, 'JupyterNB')

['Covid19', 'PoweredOffVMs']


### Insert one document

Connect to the docker mongodb and insert one document in the database JupyterNB, collection PoweredOffVMs

In [ ]:
def add_document(connectionstring, mongodatabase, collection, document):
    '''Connect to MongoDB and add a document'''
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[mongodatabase]
    mongo_col    = mongo_db[collection]
    mongo_col.insert_one(document)
    for doc in mongo_col.find():
        print(doc)

    mongo_client.close()

connectionstring = "mongodb://0.0.0.0:80"
add_document(connectionstring, 'JupyterNB', 'PoweredOffVMs', { "PSComputerName": "xsc0000991.child.domain.com", "PowerState": "Off" })

### List all documents

Connect to the docker mongodb and list all documents from a collection

In [ ]:
def get_documents(connectionstring, database, collection):
    '''Connect to MongoDB and return data'''
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[database]
    mongo_col    = mongo_db[collection]
    mongo_client.close()
    return mongo_col.find()

import pandas as pd
table = []
connectionstring = "mongodb://0.0.0.0:80"
result = get_documents(connectionstring, 'JupyterNB', 'PoweredOffVMs')
for doc in result:
    #print(doc)
    thisdict = { 
        'PSComputerName' : doc['PSComputerName'],
        'PowerState'     : doc['PowerState'],
    }
    table.append(thisdict)

df = pd.DataFrame(table)
df.head()

### Delete all documents

Connect to the docker mongodb and delete all documents from collection

In [ ]:
def delete_collection(connectionstring, mongodatabase, collection):
    '''Connect to MongoDB and add a document'''
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[mongodatabase]
    mongo_col    = mongo_db[collection]
    mongo_col.delete_many({})
    for doc in mongo_col.find():
        print(doc)

    mongo_client.close()

connectionstring = "mongodb://0.0.0.0:80"
delete_collection(connectionstring,'JupyterNB', 'PoweredOffVMs')

### Import from a backup

Create a backup file (not ready)

In [ ]:
def run_backup(connectionstring, database, collection, backup_dir):
    '''Connect to MongoDB and return data'''
    import pymongo
    import dns
    from bson.json_util import dumps
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[database]
    mongo_col    = mongo_db[collection]
    mongo_client.close()
    cursor = mongo_col.find({})
    file = open(backup_dir + collection +".json", "w")
    file.write('[')
    for document in cursor:
        file.write(dumps(document))
        file.write(',')
    file.write(']')
    mongo_client.close()


database         = 'JupyterNB'
collection       = 'FailedPatching'
backup_dir       = '/Users/Tinu/Downloads/mongo-backup/'

credentials = input('user:password')
connectionstring = "mongodb+srv://"+credentials+"@cluster0.epl3x.mongodb.net/?retryWrites=true&w=majority"

run_backup(connectionstring, database, collection, backup_dir)

Import from a csv-file

In [3]:
def import_csv_mongodb(file, connectionstring, mongodatabase, collection):
    '''Import-CSV in the MongoDB Atlas Collection PoweredOffVMs'''
    import csv, sys, os, pymongo
    check_path = os.path.exists(file)
    if(check_path == True):
        
        mongo_client = pymongo.MongoClient(connectionstring)
        mongo_db     = mongo_client[mongodatabase]
        mongo_col    = mongo_db[collection]

        with open(file, mode='r', newline='') as csv_file:
            csv_reader = csv.DictReader(csv_file, delimiter=';')
            try:
                mongo_col.insert_many(csv_reader)

            except csv.Error as e:
                sys.exit('file {}, line {}: {}'.format(file, csv_reader.line_num, e))

        print(mongo_db.list_collection_names())
        mongo_client.close()

    else:
        print(f'File not found: {file}')


database         = 'JupyterNB'
collection       = 'Covid19'
file_to_import   = '/Users/Tinu/Downloads/mongo-backup/Covid19.csv'
connectionstring = "mongodb://0.0.0.0:80"
import_csv_mongodb(file_to_import, connectionstring, database, collection)

List all documents

In [8]:
def get_collection(connectionstring, database, collection):

    '''Connect to MongoDB and return data'''
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[database]
    mongo_col    = mongo_db[collection]
    mongo_client.close()
    return mongo_col.find()


import pandas as pd
import matplotlib
import datetime

result = []
cconnectionstring = "mongodb://0.0.0.0:80"
for doc in get_collection(connectionstring, 'JupyterNB', 'Covid19'):
    thisdict = {                                
            'Datum'             : datetime.datetime.strptime(doc['Datum'], '%Y-%m-%d'),
            'Neue Fälle'        : int(doc['Neue Fälle']),
            'Hospitalisationen' : int(doc['Hospitalisationen']),
            'Todesfälle'        : int(doc['Todesfälle']),
    }
    result.append(thisdict)

# create a data frame set
df = pd.DataFrame(result)
df.head()

,Datum,Neue Fälle,Hospitalisationen,Todesfälle
0,2021-03-23,1844,74,11
1,2021-03-24,2022,83,20
2,2021-03-25,2046,74,69
3,2021-03-26,2053,50,10


## Interactive docker terminal

Start an interactive docker terminal to manage MongoDB. Copy the code into a shell and press enter:

```
docker exec -it mongodb bash
```

## Mongo shell

[Mongo Shell Quick Reference](https://docs.mongodb.com/manual/reference/mongo-shell/)

```
mongo
```
```
show dbs
show collections
show users
```